# Image Classification Model - MONAI

In [1]:
import torch
import monai
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    LoadImaged,
    RandSpatialCrop,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    ToTensor,
)
from monai.networks.nets import DenseNet201
from monai.networks.blocks import Convolution
from monai.losses import DiceLoss
from monai.metrics import ROCAUCMetric
from monai.data import CacheDataset, DataLoader, Dataset, ImageReader, MetaTensor
from monai.inferers import sliding_window_inference
from monai.transforms import Activations, AsDiscrete, KeepLargestConnectedComponent
from monai.visualize import plot_2d_or_3d_image
monai.utils.set_determinism(17)

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os

from skimage.io import imread, imsave, imshow

from tqdm import tqdm

import datetime
import numpy as np

##### Hyperparameters

In [2]:
batch_size = 4
num_workers = 4
learning_rate = 5e-6
num_epochs = 7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


##### Dataset Creation

In [3]:
def class_encoding(label):
    label_to_index = {
        'PDAC' : 0,
        'Melanoma' : 1,
        'CTCL' : 2,
        'Basal Cell' : 3
    }
    
    return torch.tensor(torch.nn.functional.one_hot(torch.tensor(label_to_index[label]), 4))
    
def class_decoding(index):
    index_to_label = {
        0 : 'PDAC',
        1 : 'Melanoma',
        2 : 'CTCL',
        3 : 'Basal Cell'
    }
    
    return index_to_label[torch.argmax(index).item()]
    
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, mode='train'):
        self.data_dir = os.path.join(data_dir, mode)
        self.class_dirs = os.listdir(self.data_dir)
        self.n_classes = len(self.class_dirs)
        self.image_files = []
        self.labels = []
        for label in self.class_dirs:
            temp_path = os.path.join(self.data_dir, label)
            image_paths = os.listdir(temp_path)
            self.labels.extend([class_encoding(label)]*len(image_paths))
            for image_path in image_paths:
                self.image_files.append(torch.tensor(imread(os.path.join(temp_path, image_path))))

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, index):
        image = self.image_files[index].unsqueeze(0).to(torch.float32)
        label = self.labels[index].to(torch.float32)
        return image, label

In [4]:
DATA_DIR = '../data/classification_dataset/'
train_dataset = ClassificationDataset(DATA_DIR, mode='train')
val_dataset = ClassificationDataset(DATA_DIR, mode='val')
test_dataset = ClassificationDataset(DATA_DIR, mode='test')

C:\Users\gopik\AppData\Local\Temp\ipykernel_19700\1041114759.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.nn.functional.one_hot(torch.tensor(label_to_index[label]), 4))


In [5]:
train_dataset.__len__(), val_dataset.__len__(), test_dataset.__len__()

(128, 16, 16)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, pin_memory=torch.cuda.is_available())
val_loader = DataLoader(val_dataset, batch_size=4, pin_memory=torch.cuda.is_available())
test_loader = DataLoader(test_dataset, batch_size=4, pin_memory=torch.cuda.is_available())

In [7]:
len(train_loader), len(val_loader), len(test_loader)

(32, 4, 4)

##### Model Definition - DenseNet

In [8]:
in_channels = 1
out_channels = 4
model = DenseNet201(
    spatial_dims=2, 
    in_channels=in_channels, 
    out_channels=out_channels, 
    init_features=64, 
    growth_rate=32, 
    block_config=(6, 12, 64, 48), 
    pretrained=True, 
    progress=True).to(device)
print(model)

DenseNet201(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchN

##### Loss Function and Optimizer

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

##### Training and Validation Loop

In [10]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0.0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        # Compute the loss
        loss = criterion(outputs, masks)
        # Backward pass and optimization
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)

    # Evaluate on the test set
    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass
            outputs = model(images)

            # Compute the loss
            loss = criterion(outputs, masks)

            test_loss += loss.item()

        test_loss /= len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

 14%|█▍        | 1/7 [00:19<01:56, 19.42s/it]

Epoch 1/7, Train Loss: 0.3404, Test Loss: 0.3477


 29%|██▊       | 2/7 [00:38<01:35, 19.07s/it]

Epoch 2/7, Train Loss: 0.3230, Test Loss: 0.3268


 43%|████▎     | 3/7 [00:56<01:13, 18.50s/it]

...

Epoch 5/7, Train Loss: 0.2754, Test Loss: 0.2896


 86%|████████▌ | 6/7 [01:51<00:18, 18.57s/it]

Epoch 6/7, Train Loss: 0.2608, Test Loss: 0.2927


100%|██████████| 7/7 [02:11<00:00, 18.85s/it]

Epoch 7/7, Train Loss: 0.2517, Test Loss: 0.2743


In [11]:
# Save the trained model
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_name = f"classification_model_monai_densenet_{timestamp}.pth"
save_path = f"../models/classification_model/{model_name}"
torch.save(model.state_dict(), save_path)

##### Testing

In [25]:
metric = torch.nn.CrossEntropyLoss()
threshold_fn = AsDiscrete(threshold=0.5)

In [26]:
model.eval()
with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = threshold_fn(Activations(sigmoid=True)(model(images)))
        loss = metric(outputs, masks)
loss_score = loss.item()/len(test_loader.dataset)
print(f"Test Loss: {loss_score}")

Test Loss: 0.06614859402179718


##### Visualization

In [27]:
image, label = test_dataset.__getitem__(10)

In [28]:
imshow(image.squeeze().numpy(), cmap="gray")

In [30]:
sam_output = model(image.unsqueeze(0).to(device))

In [33]:
class_decoding(threshold_fn(Activations(sigmoid=True)(sam_output)).clone().detach())

'Melanoma'

In [36]:
class_decoding(label)

'Melanoma'